In [ ]:
import operator as op
import os
from itertools import groupby

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import yaml
from IPython.display import Image
from matplotlib.patches import Rectangle
from shapely.geometry import LineString, Point

from config import (HEAD_TO_TAIL_JSON, SPECS_COMPLETE_YAMLS,
                    USA_STATES_ZIP, MEX_STATES_ZIP,
                    SP_RESULTS_DIR, RO_RESULTS_DIR, FIG_RESULTS_DIR)
from drawutil import rfcolors, mpl_config
from model import Solution

In [ ]:
# global setting
SCALE = 1 # scale = 1.0 for write-up dimensions
mpl_config(SCALE)

In [ ]:
colors = dict(enumerate(rfcolors))

In [ ]:
f = 62
r_max = 3
casestudy = 'harvey'
pftype = 'dc'
approach = 'stochastic'
timelimit = 45 * 60
viz = 'mitigation'
#viz = 'flooding'

In [ ]:
if approach == 'stochastic':
    MY_RESULTS_DIR = SP_RESULTS_DIR
elif approach == 'robust':
    MY_RESULTS_DIR = RO_RESULTS_DIR
else:
    raise ValueError('`approach` must be either "stochastic" or "robust"')

In [ ]:
# load the instance data
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    specs['options']['approach'] = approach
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_max)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_max:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_max) + 1)]
    c = pd.Series(specs['c'])
    c.index.names = ['k', 'r']
    xi = pd.Series(specs['xi'])
    xi.index.names = ['k', 'r', 'omega']
    r_hat = pd.Series(specs['r_hat'])
    r_hat.index.names = ['k']
    K = sorted(specs['K'])
    K_flood = sorted(set(k for (k, r, omega) in specs['xi']))
    K_exorable = set()
    xi_sorted = sorted(specs['xi'].keys(), key=lambda x: (x[2], x[0], x[1]))
    for omega, grp1 in groupby(xi_sorted, key=lambda x: x[2]):
        for k, grp2 in groupby(grp1, key=lambda x: x[0]):
            rs = [r for (_, r, _) in grp2]
            if r_hat[k] not in rs:
                K_exorable.add(k)
    K_exorable = sorted(K_exorable)
    K_load = set()
    for n in specs['N']:
        if n in specs['D_n']:
            K_load.add(specs['k_of_n'][n])
    K_load = sorted(K_load)

# load the topological data
usa = gpd.read_file(USA_STATES_ZIP)
mex = gpd.read_file(MEX_STATES_ZIP)
nodes_filename = os.path.join(HEAD_TO_TAIL_JSON)
nodes = gpd.read_file(nodes_filename)\
           .drop(columns=['tail_node'])\
           .rename(columns={'head_node': 'k'})\
           .set_index('k')
nodes['geometry'] = nodes['geometry'].apply(lambda linestring: Point(linestring.coords[1]))
nodes = nodes.loc[nodes.index.isin(K)]
nodes.crs = 'epsg:4326'

In [ ]:
if viz == 'mitigation':
    # load the solution data
    zipfile = os.path.join(MY_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_max}.zip')
    solution = Solution.from_zip(zipfile)
    sol_x = solution['x'].round().copy()
    sol_x.index.names = ['k', 'r']
    sol_x = sol_x.unstack('r')\
                 .reindex(nodes.index, axis=0)\
                 .reindex([0, 1, 2, 3], axis=1)\
                 .fillna(0)\
                 .astype(int)
    sol_x = sol_x.multiply(sol_x.columns, axis=1)\
                 .max(axis=1)
    # add solution data to topological data
    nodes['level'] = sol_x.copy()
    nodes['size'] = c.groupby(c.index.get_level_values('k')).min()
    nodes['marker'] = 'o'
    nodes.loc[K_load, 'marker'] = 'v'
    nodes['markercolor'] = nodes['level'].apply(colors.get)
    nodes['markersize'] = nodes['size'].apply(lambda x: 15 + (x - 1) * 15)
    # filter nodes data
    nodes = nodes.loc[nodes.index.isin(K_exorable)]
if viz == 'flooding':
    # add flood levels to topological data
    xi_tmp = xi.unstack(['omega', 'r']).fillna(0)
    xi_tmp = xi_tmp.multiply(xi_tmp.columns.get_level_values('r'), axis=1).max(axis=1)
    nodes['level'] = xi_tmp.copy()
    nodes['size'] = c.groupby(c.index.get_level_values('k')).min()
    nodes['marker'] = 'o'
    nodes.loc[K_load, 'marker'] = 'v'
    nodes['markercolor'] = nodes['level'].apply(colors.get)
    nodes['markersize'] = nodes['size'].apply(lambda x: 15 + (x - 1) * 15)
    # filter nodes data
    nodes = nodes.loc[nodes.index.isin(K_flood)]

In [ ]:
# overall size of figure
figsize = (3, 3.5)

# x and y ranges of the main vizualization and the insets
#insets = ['a', 'b'] if casestudy == 'harvey' else []
insets = ['a'] if casestudy == 'harvey' else []
limits = {
    'M': {'x': [-98.350, -94.350], 'y': [26.000, 30.667]},
    #'a': {'x': [ -95.700, -94.800], 'y': [  28.900,  30.05]},
    'a': {'x': [ -95.700, -94.800], 'y': [  29.50,  30.05]},
    'b': {'x': [ -97.87, -97.72], 'y': [  27.25,  27.40]},
}

# scaling of each ax
y_to_x = {
    inset: op.sub(*limits[inset]['y']) / op.sub(*limits[inset]['x'])
    for inset in ['M'] + insets
}

# position of main ax
positions = dict()
positions['M'] = [0.00, 0.00, 1.00, 1.00]

# margin sizes of inset ax relative to main ax
y_margin = 0.02
x_margin = y_margin * y_to_x['M']

# position for inset a
zoom = 2.5
x_dim = -op.sub(*limits['a']['x']) / -op.sub(*limits['M']['x']) * zoom
y_dim = -op.sub(*limits['a']['y']) / -op.sub(*limits['M']['y']) * zoom
positions['a'] = [1 - x_margin - x_dim, y_margin, x_dim, y_dim]

# position for inset b
zoom = 4.0
x_dim = -op.sub(*limits['b']['x']) / -op.sub(*limits['M']['x']) * zoom
y_dim = -op.sub(*limits['b']['y']) / -op.sub(*limits['M']['y']) * zoom
positions['b'] = [positions['a'][0] - 1 * x_margin - x_dim, 10 * y_margin, x_dim, y_dim]

In [ ]:
# ax is made to be the main axis, and axes are for the insets
if insets:
    fig, (ax, *_axes) = plt.subplots(1, 1 + len(insets), figsize=figsize)
    axes = dict(zip(insets, _axes))
else:
    fig, ax = plt.subplots(1, 1, figsize=figsize)


ax.set_position(positions['M'])
ax.set_facecolor('lightblue')
usa.plot(ax=ax, edgecolor='black', facecolor='#6f6f6f')
mex.plot(ax=ax, edgecolor='black', facecolor='#6f6f6f')
          
for (markersize, marker), grp in nodes.groupby(['markersize', 'marker']):
    grp.plot(ax=ax,
             marker=marker,
             markersize=markersize,
             color=grp['markercolor'],
             edgecolor='black',
             linewidth=0.50,
             zorder=5-markersize/15)
ax.axes.set_aspect('auto')  # dangerous; scale is determined by user inputs
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(limits['M']['x'])
ax.set_ylim(limits['M']['y'])
for key in ax.spines.keys():
    ax.spines[key].set_linewidth(1.50)

for inset in insets:
    # make inset
    axi = axes[inset]
    axi.set_position(positions[inset])
    axi.set_facecolor('lightblue')
    usa.plot(ax=axi, edgecolor='black', facecolor='#6f6f6f')
    mex.plot(ax=axi, edgecolor='black', facecolor='#6f6f6f')
    for (markersize, marker), grp in nodes.groupby(['markersize', 'marker']):
        grp.plot(ax=axi,
                 marker=marker,
                 markersize=markersize,
                 color=grp['markercolor'],
                 edgecolor='black',
                 linewidth=0.50,
                 zorder=5-markersize/15)
    axi.axes.set_aspect('auto')
    axi.set_xticks([])
    axi.set_yticks([])
    xlim = limits[inset]['x']
    ylim = limits[inset]['y']
    axi.set_xlim(xlim)
    axi.set_ylim(ylim)
    for key in axi.spines.keys():
        axi.spines[key].set_linewidth(1.50)
    # create dotted box and guidelines in main image
    rect = Rectangle((xlim[0], ylim[0]), xlim[1] - xlim[0], ylim[1] - ylim[0],
                     edgecolor='black', linestyle='--', linewidth=1.00,
                     facecolor='#ffffff5f', zorder=10)
    ax.add_patch(rect)

if casestudy == 'harvey' and 'a' in insets:
    # guidelines
    from1 = (limits['a']['x'][0],
             limits['a']['y'][1])
    to1 = (limits['M']['x'][0] + (limits['M']['x'][1] - limits['M']['x'][0]) * (positions['a'][0]),
           limits['M']['y'][0] + (limits['M']['y'][1] - limits['M']['y'][0]) * (positions['a'][1] + positions['a'][3]))
    from2 = (limits['a']['x'][1],
             limits['a']['y'][1])
    to2 = (limits['M']['x'][0] + (limits['M']['x'][1] - limits['M']['x'][0]) * (positions['a'][0] + positions['a'][2]),
           limits['M']['y'][0] + (limits['M']['y'][1] - limits['M']['y'][0]) * (positions['a'][1] + positions['a'][3]))
    ax.plot(*zip(from1, to1), 'k--', linewidth=1.00)
    ax.plot(*zip(from2, to2), 'k--', linewidth=1.00)

if casestudy == 'harvey' and 'b' in insets:
    # guidelines
    from1 = (limits['b']['x'][0],
             limits['b']['y'][0])
    to1 = (limits['M']['x'][0] + (limits['M']['x'][1] - limits['M']['x'][0]) * (positions['b'][0]),
           limits['M']['y'][0] + (limits['M']['y'][1] - limits['M']['y'][0]) * (positions['b'][1]))
    from2 = (limits['b']['x'][0],
             limits['b']['y'][1])
    to2 = (limits['M']['x'][0] + (limits['M']['x'][1] - limits['M']['x'][0]) * (positions['b'][0]),
           limits['M']['y'][0] + (limits['M']['y'][1] - limits['M']['y'][0]) * (positions['b'][1] + positions['b'][3]))
    ax.plot(*zip(from1, to1), 'k--', linewidth=1.00)
    ax.plot(*zip(from2, to2), 'k--', linewidth=1.00)


# legend
ax.plot([0], [0], 'kv', markersize=3, label='Load')
ax.plot([0], [0], 'ko', markersize=3, label='No Load')

if viz == 'mitigation':
    for _r in range(0, r_max):
        ax.plot([0], [0], 's', markersize=3, color=colors[_r], label=f'$r={_r}$')
elif viz == 'flooding':
    for _r in range(1, r_max + 1):
        ax.plot([0], [0], 's', markersize=3, color=colors[_r], label=f'$r={_r}$')
legend = ax.legend(loc='upper left',
                   edgecolor='black',
                   facecolor='white',
                   fancybox=False,
                   handletextpad=0.25,
                   handlelength=1,
                   labelspacing=0.2)

if viz == 'mitigation':
    filename = f'ijoc-mitigation-{casestudy}-{pftype}-{approach}-r{r_max}-f{f}.jpg'
if viz == 'flooding':
    filename = f'ijoc-flooding-{casestudy}.jpg'

filename = os.path.join(FIG_RESULTS_DIR, filename)
plt.savefig(filename, format='jpg', dpi=256)
plt.close()
display(Image(filename))